In [1]:
import numpy as np

## LM Head Forwording

Mathemetic definition

$$Logits = h_{last} \cdot W_{vocab} + b_{vocab}$$

Where 
* $h_{last} \in R^{(d_{model}, )}$
* $W_{vocab} \in R^{(d_{model} \times vocab\_size)}$
* $b_{vocab} \in R^{(vocab\_size,)}$ 
* $logits \in R^{(vocab\_size,)}$ 

$h_{last}$ คือ hidden state ของ token ตัวสุดท้ายที่ได้จาก stack ของ transformer 

In [2]:
d_model = 6
vocab_size = 8
V = vocab_size
T = 4


In [3]:
W_vocab = np.random.randn(d_model, V)
b_vocab = np.random.randn(vocab_size)
h = np.random.rand(T, d_model)

print(h)

[[0.09209689 0.18642851 0.67247477 0.38973948 0.92231518 0.11290127]
 [0.63198737 0.19525941 0.86433159 0.9339583  0.25040326 0.96829935]
 [0.75501455 0.77119413 0.13094035 0.62081439 0.99561573 0.00617687]
 [0.99952434 0.37665087 0.6671144  0.13086864 0.16358304 0.42539695]]


In [4]:
logits = h @ W_vocab + b_vocab
print(logits)

[[ 1.70806142 -0.48427631  1.76560427  0.68417436 -1.46413508 -0.27127701
   2.23593551  2.13306633]
 [ 4.24193215 -0.94924942  2.69494946 -0.74614866 -1.53308525 -1.12118069
   2.49544409  3.06266967]
 [ 2.06769906 -2.0824944   2.49326643  1.93357135 -1.25968496  0.19006748
   1.31715957  3.24419259]
 [ 2.28746241 -1.37434603  2.80411671  0.38247886 -0.96567009 -0.04252615
   2.27641777  2.68525429]]


### In case of many batch

In [5]:
B = 2 # batch size

In [6]:
h = np.random.rand(B, T, d_model)
print (h.shape)

(2, 4, 6)


In [7]:
W_vocab.shape

(6, 8)

In [8]:
h[:, -1, :].shape

(2, 6)

In [9]:
logits = (h[:, -1, :] @ W_vocab) + b_vocab
print("logits => \n")
print(logits)

logits => 

[[ 2.31387085 -1.66742949  2.8275622   1.48686371 -1.45177755  0.20367131
   1.81308598  3.53450131]
 [ 2.46536495 -1.72016642  1.62665673  0.28920771 -1.13115772 -0.30036748
   1.43114085  2.24742319]]


## Back propagation of LM head

from the last chapter (loss And Softmax)

เราทราบว่า

$$\frac {\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} = \frac {\partial \mathcal {L}} {\partial Z} \in R^{B \times T \times V}$$

หรือมีมิติเท่ากับ logits

เมื่อ 
* $\mathcal {L}$ คือ loss ของ model
* $p$ คือ ความน่าจะเป็นของแต่ละคำที่จะออกเป็น token ถัดไป 
* $Z$ คือ Logits ที่ได้จากการคำนวณด้วย LM Head

In [12]:
dZ = np.random.rand(B, T, V)
print("dZ =>", dZ.shape)

dZ => (2, 4, 8)


### Start from math
---

#### เมื่อต้องการปรับค่า $W_{vocab}$ เพื่อให้ loss มีค่าลดลงจะต้องปรับโดยการ : 
$$ W^{new}_{vocab} = W^{old}_{vocab} - \alpha \frac{\partial \mathcal {L}} {\partial W_{vocab}}$$

จาก Chain rule : 

$$\frac{\partial \mathcal {L}} {\partial W_{vocab}} = \frac{\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} \cdot \frac{\partial Z} {\partial W_{vocab}}$$

ซึ่ง 

$$ \frac{\partial Z} {\partial W_{vocab}} = h_{last\ token}$$

ได้ว่า

$$ W^{new}_{vocab} = W^{old}_{vocab} - \alpha \frac{\partial \mathcal {L} } {\partial Z} \cdot h_{last\ token}$$

เมื่อ $\alpha$ คือ learning rate

#### เมื่อต้องการปรับค่า $b_{vocab}$ เพื่อให้ loss มีค่าลดลงจะต้องปรับโดยการ : 

$$ b^{new}_{vocab} = b^{old}_{vocab} - \alpha \frac{\partial \mathcal {L}} {\partial b_{vocab}}$$

จาก Chain rule : 

$$\frac{\partial \mathcal {L}} {\partial b_{vocab}} = \frac{\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} \cdot \frac{\partial Z} {\partial b}$$

ซึ่ง 

$$ \frac{\partial Z} {\partial b_{vocab}} = 1$$

ได้ว่า

$$ b^{new}_{vocab} = b^{old}_{vocab} - \alpha \frac{\partial \mathcal {L} } {\partial Z}$$

### Code demo
---

In [14]:
alpha = 0.1


In [15]:
print(dZ.shape, h.shape)

(2, 4, 8) (2, 4, 6)


In [20]:
h.T @ dZ

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 2)

In [16]:
W_vocab.shape

(6, 8)

In [ ]:
W_vocab_new = W_vocab - alpha * 